# Task for tomorrow:
5/5/2025: 
* Bắt đầu làm PCA (không chắc là có cần PCA hay không)
* Chạy thử Random Forest
* Cân nhắc 2 hướng: Split 30% data thành training và testing set + 30% để training hết (white-list) và sau đó dùng 70% để testing lun


# Import packages and Dataset

In [33]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [34]:
swisstools_assets = r"C:\Users\iamti\qa\swisstools_assets.csv"
swisstools_asset_types = r"C:\Users\iamti\qa\swisstools_asset_types.csv"
swisstools_asset_properties = r"C:\Users\iamti\qa\swisstools_asset_properties.csv"
swisstools_asset_relationships = r"C:\Users\iamti\qa\swisstools_asset_relationships.csv"
swisstools_asset_relationship_services = r"C:\Users\iamti\qa\swisstools_asset_relationship_services.csv"
swisstools_asset_relationship_service_metrics = r"C:\Users\iamti\qa\swisstools_asset_relationship_service_metrics.csv"

haarlem_assets = r"C:\Users\iamti\qa\haarlem_assets.csv"
haarlem_asset_types = r"C:\Users\iamti\qa\haarlem_asset_types.csv"
haarlem_asset_properties = r"C:\Users\iamti\qa\haarlem_asset_properties.csv"
haarlem_asset_relationships = r"C:\Users\iamti\qa\haarlem_asset_relationships.csv"
haarlem_asset_relationship_services = r"C:\Users\iamti\qa\haarlem_asset_relationship_services.csv"
haarlem_asset_relationship_service_metrics = r"C:\Users\iamti\qa\haarlem_asset_relationship_service_metrics.csv"

backend_assets = r"C:\Users\iamti\qa\backend_asset.csv"
backend_asset_types = r"C:\Users\iamti\qa\backend_assettype.csv"
backend_asset_properties = r"C:\Users\iamti\qa\backend_assetproperty.csv"
backend_asset_relationships = r"C:\Users\iamti\qa\backend_assetrelationship.csv"
backend_asset_relationship_services = r"C:\Users\iamti\qa\backend_assetrelationshipservice.csv"
backend_asset_relationship_service_metrics = r"C:\Users\iamti\qa\backend_assetrelationshipservicemetric.csv" 

s_assets = pd.DataFrame(pd.read_csv(swisstools_assets))
s_types = pd.DataFrame(pd.read_csv(swisstools_asset_types))
s_properties = pd.DataFrame(pd.read_csv(swisstools_asset_properties))
s_rela = pd.DataFrame(pd.read_csv(swisstools_asset_relationships))
s_services = pd.DataFrame(pd.read_csv(swisstools_asset_relationship_services))
s_metrics = pd.DataFrame(pd.read_csv(swisstools_asset_relationship_service_metrics))

h_assets = pd.DataFrame(pd.read_csv(haarlem_assets))
h_types = pd.DataFrame(pd.read_csv(haarlem_asset_types))
h_properties = pd.DataFrame(pd.read_csv(haarlem_asset_properties))
h_rela = pd.DataFrame(pd.read_csv(haarlem_asset_relationships))
h_services = pd.DataFrame(pd.read_csv(haarlem_asset_relationship_services))
h_metrics = pd.DataFrame(pd.read_csv(haarlem_asset_relationship_service_metrics))

b_assets = pd.DataFrame(pd.read_csv(backend_assets))
b_types = pd.DataFrame(pd.read_csv(backend_asset_types))
b_properties = pd.DataFrame(pd.read_csv(backend_asset_properties))
b_rela = pd.DataFrame(pd.read_csv(backend_asset_relationships))
b_services = pd.DataFrame(pd.read_csv(backend_asset_relationship_services))
b_metrics = pd.DataFrame(pd.read_csv(backend_asset_relationship_service_metrics))

C:\Users\iamti\AppData\Local\Temp\ipykernel_92800\906210989.py:22: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  s_assets = pd.DataFrame(pd.read_csv(swisstools_assets))
C:\Users\iamti\AppData\Local\Temp\ipykernel_92800\906210989.py:36: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  b_assets = pd.DataFrame(pd.read_csv(backend_assets))


# Define the Prediction Target
* Target variable: type.name
* Number of haarlem_types.name values starting with 'Unknown asset': 23/105. This is not the case for swiss. Both have no missing value.

# Cleaning Data
Merge important variables into Asset table and drop those unnecessary
FACT CHECK THIS: For each id, they only act as either sender or receiver for all services.

TEST CÓ BYTE VÀ KHÔNG CÓ BYTE + TEST BYTE ĐƯỢC CỘNG DỒN VÀ BYTE KHÔNG CỘNG DỒN


## Swiss Dataset

In [ ]:
# AGGREGATE BYTES 
# Mai check lại thử xem mọi thứ có make sense không nhé

original_row_count = len(s_assets)

asset_merged = s_assets.merge(
    s_types[['id', 'category', 'name']],
    left_on='asset_type_id',
    right_on='id',
    how='left'  # keep all rows from asset_df, even if no matching type
)
asset_merged = asset_merged.rename(columns={
    'id_x': 'asset_id',         # asset ID
    'id_y': 'type_id',    # the same as asset_type_id
    'name_y': 'asset_type_name' # name of asset_type_id
})
rel_service_merged  = s_rela.merge(
    s_services[['relationship_id', 'protocol', 'bytes', 'count']],
    left_on='id',
    right_on='relationship_id',
    how='right'
)

bytes_sent_df = (rel_service_merged.groupby('source_asset_id')['bytes']
    .sum().reset_index().rename(columns={'source_asset_id': 'asset_id', 'bytes': 'total_bytes_sent'}))
bytes_recv_df = (rel_service_merged.groupby('target_asset_id')['bytes']
    .sum().reset_index().rename(columns={'target_asset_id': 'asset_id', 'bytes': 'total_bytes_received'}))
count_sent_df = (rel_service_merged.groupby('source_asset_id')['count']
    .sum().reset_index().rename(columns={'source_asset_id': 'asset_id', 'count': 'total_count_sent'}))
count_recv_df = (rel_service_merged.groupby('target_asset_id')['count']
    .sum().reset_index().rename(columns={'target_asset_id': 'asset_id', 'count': 'total_count_received'}))
all_protocols = pd.concat([
    rel_service_merged[['source_asset_id', 'protocol']].rename(columns={'source_asset_id': 'asset_id'}),
    rel_service_merged[['target_asset_id', 'protocol']].rename(columns={'target_asset_id': 'asset_id'})
])
protocols_df = (
    all_protocols.groupby('asset_id')['protocol']
    .apply(lambda x: ','.join(x.dropna().unique()))
    .reset_index()
    .rename(columns={'protocol': 'protocols_used'})
)

final_df = (
    asset_merged
    .merge(bytes_sent_df, on='asset_id', how='left')
    .merge(bytes_recv_df, on='asset_id', how='left')
    .merge(count_sent_df, on='asset_id', how='left')
    .merge(count_recv_df, on='asset_id', how='left')
    .merge(protocols_df, on='asset_id', how='left')
    .fillna({
        'total_bytes_sent': 0,
        'total_bytes_received': 0,
        'total_count_sent': 0,
        'total_count_received': 0,
        'protocols_used': ''
    })
)

protocol_dummies = final_df['protocols_used'].str.get_dummies(sep=',')
final_df = final_df.join(protocol_dummies)
final_df = final_df[[
    'asset_id', 'hostname', 'ipaddress', 'mac_address', 'vlan_ids', 'parent_asset_id',
    'asset_type_id', 'asset_type_name', 'category',
    'total_bytes_sent', 'total_bytes_received',
    'total_count_sent', 'total_count_received', 
    'protocols_used', 
    *[f'{col}' for col in protocol_dummies.columns]  # Dummy vars
]]

# display(final_df[final_df['protocols_used'] == 'UDP,TCP'])
display(final_df)

,asset_id,hostname,ipaddress,mac_address,vlan_ids,parent_asset_id,asset_type_id,asset_type_name,category,total_bytes_sent,total_bytes_received,total_count_sent,total_count_received,protocols_used,SNMP,TCP,UDP
0,eb2cb00a-5b3e-4ebb-a661-81406babfdc3,frmrs1-vip-bx-003.b.aaplimg.com.,17.253.122.200,00:09:0f:09:00:06,{45},NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,,0,0,0
1,1db9766c-2ba8-416b-85e4-7fce554424a1,NaN,59.95.128.68,f0:4a:02:65:d8:d4,{1001},NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,,0,0,0
2,8ae80eeb-fef5-4504-a720-79f474eb01aa,BENB03.sasyb.local.,192.168.103.21/24,00:09:0f:09:00:06,{2102},cbb93477-13a5-4d42-9252-b0a62639376e,12d5edc6-3b58-404e-93c4-1967b56e3c6d,Network asset,Network,0.0,10710.0,0.0,153.0,TCP,0,1,0
3,a4e93b1c-5a42-4df5-a91d-f8261e7e3379,NaN,183.160.194.143,f0:4a:02:65:d8:d4,{1001},NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,,0,0,0
4,a88fec76-bbaf-469d-8cd2-3b6b0ae8fc18,NaN,27.25.152.221,f0:4a:02:65:d8:d4,{1001},NaN,962a2cc4-e7d2-4fd8-bd43-65a1a07a9049,Network asset,Network,0.0,0.0,0.0,0.0,,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229028,46c0489e-f333-45fb-ac49-525f8c74c09b,softbank126110055194.bbtec.net.,126.110.55.194,f0:4a:02:65:d8:d4,{1001},NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,,0,0,0
229029,304e06b3-fcb8-4f9b-a1d8-c2113f1738c2,ec2-34-249-248-37.eu-west-1.compute.amazonaws....,34.249.248.37,00:09:0f:09:00:06,{45},NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,,0,0,0
229030,e2ba6437-d675-4131-abc1-5e6d135e6de4,NaN,196.251.92.50,f0:4a:02:65:d8:d4,{1001},NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,,0,0,0
229031,b4cce496-54a7-4a36-867b-fceee91c3be2,NaN,196.191.102.45,f0:4a:02:65:d8:d4,{1001},NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,,0,0,0


In [ ]:
# NON AGGREGATE BYTES
# Chưa fix lại cái này nhé, chỉ là tạm thời thôi

original_row_count = len(s_assets)

# Merge asset with type information (same as before)
asset_merged = s_assets.merge(
    s_types[['id', 'category', 'name']],
    left_on='asset_type_id',
    right_on='id',
    how='left'
).rename(columns={
    'id_x': 'id',
    'id_y': 'type_id',
    'name_y': 'type_name'
})

# Create the service-relationship merge (same as before)
service_rel = s_services.merge(
    s_rela[['id', 'source_asset_id', 'target_asset_id']],
    left_on='relationship_id',
    right_on='id',
    how='left'
)

# Create sender rows
sender_rows = service_rel.copy()
sender_rows = sender_rows.merge(
    asset_merged,
    left_on='source_asset_id',
    right_on='id',
    how='left'
)
sender_rows['bytes_sent'] = sender_rows['bytes']
sender_rows['bytes_received'] = 0
sender_rows['role'] = 'sender'

# Create receiver rows
receiver_rows = service_rel.copy()
receiver_rows = receiver_rows.merge(
    asset_merged,
    left_on='target_asset_id',
    right_on='id',
    how='left'
)
receiver_rows['bytes_sent'] = 0
receiver_rows['bytes_received'] = receiver_rows['bytes']
receiver_rows['role'] = 'receiver'

# Combine sender and receiver rows
combined = pd.concat([sender_rows, receiver_rows])

# One-hot encode protocols (only one protocol per row will be 1)
protocol_dummies = pd.get_dummies(combined['protocol'], prefix='', prefix_sep='')

# Ensure we have all three protocol columns even if some are missing
for proto in ['tcp', 'udp', 'snmp']:
    if proto not in protocol_dummies.columns:
        protocol_dummies[proto] = 0

# Select and order columns
selected_columns = [
    'id', 'hostname', 'ipaddress', 'asset_type_id',
    'mac_address', 'vlan_ids', 'parent_asset_id',
    'category', 'bytes_sent', 'bytes_received', 'role'
]
final_df = combined[selected_columns].join(protocol_dummies[['tcp', 'udp', 'snmp']])

In [141]:
display(rel_service_merged.head())

,id,tags,date_created,date_updated,source_asset_id,target_asset_id,relationship_id,protocol,bytes,count
0,a92eae18-4070-4ddd-9ed9-22b60448637f,NaN,2024-10-31 11:01:40.662803+01,2024-10-31 11:01:40.662821+01,68c06b80-b39c-40d4-828c-fdf28329df2b,a5445053-6b28-450b-9e2d-886b0cb2e7b7,a92eae18-4070-4ddd-9ed9-22b60448637f,UDP,98.0,1.0
1,1bdda16a-e369-4dc9-8e5d-b976f465e904,NaN,2024-10-09 23:53:54.45059+02,2024-10-09 23:53:54.450613+02,a551fc87-630f-4b89-8b43-4e89325c43cc,50b921a6-cba7-435a-8630-4c2b179d43b6,1bdda16a-e369-4dc9-8e5d-b976f465e904,UDP,98.0,1.0
2,a92eae18-4070-4ddd-9ed9-22b60448637f,NaN,2024-10-31 11:01:40.662803+01,2024-10-31 11:01:40.662821+01,68c06b80-b39c-40d4-828c-fdf28329df2b,a5445053-6b28-450b-9e2d-886b0cb2e7b7,a92eae18-4070-4ddd-9ed9-22b60448637f,UDP,0.0,0.0
3,a92eae18-4070-4ddd-9ed9-22b60448637f,NaN,2024-10-31 11:01:40.662803+01,2024-10-31 11:01:40.662821+01,68c06b80-b39c-40d4-828c-fdf28329df2b,a5445053-6b28-450b-9e2d-886b0cb2e7b7,a92eae18-4070-4ddd-9ed9-22b60448637f,UDP,98.0,1.0
4,1bdda16a-e369-4dc9-8e5d-b976f465e904,NaN,2024-10-09 23:53:54.45059+02,2024-10-09 23:53:54.450613+02,a551fc87-630f-4b89-8b43-4e89325c43cc,50b921a6-cba7-435a-8630-4c2b179d43b6,1bdda16a-e369-4dc9-8e5d-b976f465e904,UDP,98.0,1.0


In [236]:
# Filter for rows that have a valid target (category is not null)
ml_df = final_df[final_df['category'].notna()]

# Stricter filtering: Drop rows where ALL features are missing (e.g., no protocol, no bytes)
# ml_df = ml_df[
#     (ml_df['total_bytes_sent'] > 0) |
#     (ml_df['total_bytes_received'] > 0) 
#     # (ml_df['protocols_used'] != '')
# ]

print(f"✅ Number of rows in asset_df before merging: {original_row_count}")
print(f"✅ Number of rows in final_df before filtering: {len(final_df)}")
print(f"✅ Number of rows usable for ML (with valid category): {len(ml_df)}")

display(ml_df.head())

✅ Number of rows in asset_df before merging: 229033
✅ Number of rows in final_df before filtering: 229033
✅ Number of rows usable for ML (with valid category): 69151


,asset_id,hostname,ipaddress,mac_address,vlan_ids,parent_asset_id,asset_type_id,asset_type_name,category,total_bytes_sent,total_bytes_received,total_count_sent,total_count_received,protocols_used,SNMP,TCP,UDP
2,8ae80eeb-fef5-4504-a720-79f474eb01aa,BENB03.sasyb.local.,192.168.103.21/24,00:09:0f:09:00:06,{2102},cbb93477-13a5-4d42-9252-b0a62639376e,12d5edc6-3b58-404e-93c4-1967b56e3c6d,Network asset,Network,0.0,10710.0,0.0,153.0,TCP,0,1,0
4,a88fec76-bbaf-469d-8cd2-3b6b0ae8fc18,NaN,27.25.152.221,f0:4a:02:65:d8:d4,{1001},NaN,962a2cc4-e7d2-4fd8-bd43-65a1a07a9049,Network asset,Network,0.0,0.0,0.0,0.0,,0,0,0
5,ea8cb320-c652-4734-88ee-9bbe0b5a142b,NaN,52.123.242.153,dc:00:b0:6a:7f:dc,{1001},6df29d7c-1537-4e30-8cc4-22a5131f3f39,0c50b9d6-e84c-482c-bd79-19d83ae7db5d,Network asset,Network,0.0,0.0,0.0,0.0,,0,0,0
6,93bc12be-697b-4d05-8b32-66d085877b4e,NaN,47.97.33.235,00:09:0f:09:00:06,{120},NaN,12d5edc6-3b58-404e-93c4-1967b56e3c6d,Network asset,Network,0.0,0.0,0.0,0.0,,0,0,0
7,cf1b8c93-93a1-4fe3-95f4-eb9f2acb9ff8,NaN,40.86.155.170,f0:4a:02:65:d8:d4,{1001},NaN,962a2cc4-e7d2-4fd8-bd43-65a1a07a9049,Network asset,Network,0.0,0.0,0.0,0.0,,0,0,0


In [143]:
# ml_df[ml_df['TCP']>0]
ml_df[ml_df['category']=='Network']

,id,hostname,ipaddress,asset_type_id,mac_address,vlan_ids,parent_asset_id,category,bytes_sent,bytes_received,role,tcp,udp,snmp
1,a551fc87-630f-4b89-8b43-4e89325c43cc,FREEBOX SWT,192.168.9.100/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:08,{1007},NaN,Network,98.0,0.0,sender,0,0,0
1,a551fc87-630f-4b89-8b43-4e89325c43cc,FREEBOX SWT,192.168.9.100/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:08,{1007},NaN,Network,98.0,0.0,sender,0,0,0
4,a551fc87-630f-4b89-8b43-4e89325c43cc,FREEBOX SWT,192.168.9.100/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:08,{1007},NaN,Network,98.0,0.0,sender,0,0,0
4,a551fc87-630f-4b89-8b43-4e89325c43cc,FREEBOX SWT,192.168.9.100/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:08,{1007},NaN,Network,98.0,0.0,sender,0,0,0
6,a551fc87-630f-4b89-8b43-4e89325c43cc,FREEBOX SWT,192.168.9.100/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:08,{1007},NaN,Network,95.0,0.0,sender,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573816,b0c1b7f7-e177-441f-9c7e-141f36222c0d,NaN,104.248.92.25,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2102},NaN,Network,0.0,10544.0,receiver,0,0,0
573822,b0c1b7f7-e177-441f-9c7e-141f36222c0d,NaN,104.248.92.25,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2102},NaN,Network,0.0,7987.0,receiver,0,0,0
573822,b0c1b7f7-e177-441f-9c7e-141f36222c0d,NaN,104.248.92.25,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2102},NaN,Network,0.0,7987.0,receiver,0,0,0
573831,b0c1b7f7-e177-441f-9c7e-141f36222c0d,NaN,104.248.92.25,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2102},NaN,Network,0.0,7921.0,receiver,0,0,0


## Backend Dataset

In [38]:
# Original row count
original_row_count = len(b_assets)

# Merge category and name from type table into asset table
asset_merged = b_assets.merge(
    b_types[['id', 'category', 'name']],
    left_on='asset_type_id',
    right_on='id',
    how='left'  # keep all rows from asset_df, even if no matching type
)

# Clean up merged table
asset_merged = asset_merged.rename(columns={
    'id_x': 'id',         # asset ID
    'id_y': 'type_id',    # type table ID, renamed to avoid confusion
    'name_y': 'type_name' # type.name, optional for exploration
})

# Merge protocol and bytes info from service table (aggregated per asset)
# First, merge service with relationship to map services to asset IDs
service_rel = b_services.merge(
    b_rela[['id', 'source_asset_id', 'target_asset_id']],
    left_on='relationship_id',
    right_on='id',
    how='left'
)

# Aggregate bytes sent per source asset
bytes_sent_df = service_rel.groupby('source_asset_id')['bytes'].sum().reset_index()
bytes_sent_df = bytes_sent_df.rename(columns={
    'source_asset_id': 'id',
    'bytes': 'total_bytes_sent'
})

# Aggregate bytes received per target asset
bytes_recv_df = service_rel.groupby('target_asset_id')['bytes'].sum().reset_index()
bytes_recv_df = bytes_recv_df.rename(columns={
    'target_asset_id': 'id',
    'bytes': 'total_bytes_received'
})

# Aggregate all protocols used per source asset
protocols_df = service_rel.groupby('source_asset_id')['protocol']\
    .apply(lambda x: ','.join(x.dropna().unique())).reset_index()
protocols_df = protocols_df.rename(columns={
    'source_asset_id': 'id',
    'protocol': 'protocols_used'
})

# Merge bytes sent and received into asset_merged
final_df = asset_merged.merge(bytes_sent_df, on='id', how='left')
final_df = final_df.merge(bytes_recv_df, on='id', how='left')
final_df = final_df.merge(protocols_df, on='id', how='left')
# Fill NaNs in numeric byte columns with 0
final_df[['total_bytes_sent', 'total_bytes_received']] = final_df[
    ['total_bytes_sent', 'total_bytes_received']
].fillna(0)
final_df['protocols_used'] = final_df['protocols_used'].fillna('')

# One-hot encode protocols_used
protocol_dummies = final_df['protocols_used'].str.get_dummies(sep=',')

# Merge back into final_df and drop original protocols_used column
final_df = final_df.drop(columns=['protocols_used']).join(protocol_dummies)

# Keep only relevant features
selected_columns = [
    'id', 'hostname', 'ipaddress', 'asset_type_id',
    'mac_address', 'vlan_ids', 'parent_asset_id',
    'category', 'total_bytes_sent', 'total_bytes_received'
]
final_df = final_df[selected_columns + list(protocol_dummies.columns)]


In [39]:
print(f"✅ Number of rows in asset_df before merging: {original_row_count}")
print(f"✅ Number of rows in final_df after merging: {len(final_df)}")
print("\n🧾 Preview of final dataset:")
display(final_df)

✅ Number of rows in asset_df before merging: 119125
✅ Number of rows in final_df after merging: 119125

🧾 Preview of final dataset:


,id,hostname,ipaddress,asset_type_id,mac_address,vlan_ids,parent_asset_id,category,total_bytes_sent,total_bytes_received,SNMP,TCP,UDP
0,93c84f4c-e251-443e-aedb-0f6d08fbbbb0,ec2-3-230-209-14.compute-1.amazonaws.com.,192.168.102.11/24,656c360c-f7e0-4026-b421-771f273958d2,b6:b9:76:54:2d:f6,{2102},0c9d1848-0ee1-4641-a7da-2a7313c45b74,Other,25447457.0,78700306.0,0,1,1
1,1c3225eb-bd10-4065-8528-f8d5f42cff5b,NaN,192.168.45.15/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2102},1fe7e185-f0fb-44bc-a41f-9a357e3ff77f,Network,0.0,29820.0,0,0,0
2,3e5887e1-9b95-4884-91a2-e67ec01d83ba,NaN,192.168.101.138/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{120},926411cd-ed42-44e4-ab52-c8edea870c45,Network,0.0,0.0,0,0,0
3,cbc95c38-42fe-473a-bcdb-63869fb7f7d5,NaN,192.168.0.139,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2101},NaN,Network,0.0,2030.0,0,0,0
4,dc032904-bec1-4296-8f84-c06a48fbbe28,DTNBATELIER.sasyb.local.,192.168.103.136/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{120},a4c6b641-dcf8-4a1f-a4c6-4b68aac086b8,Network,29788.0,171664.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119120,f5efb9ff-9a7c-41de-a527-8fbd8e127f23,NaN,192.168.103.105/24,c8746580-1199-41c3-a659-4870b6d265c9,f8:0d:60:9a:18:10,{2103},NaN,OT,122329.0,9304.0,0,1,1
119121,864c7d2e-6fbb-41e7-a140-b23c4636d789,NaN,10.61.242.238,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2102},NaN,Network,0.0,1580.0,0,0,0
119122,8bba5974-b250-4b2c-8ba4-53e6b96cfecb,NaN,192.168.1.28,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{45},NaN,Network,0.0,3150.0,0,0,0
119123,a655da44-5ea3-45f8-838b-b55d65a3197d,NaN,192.168.36.17,962a2cc4-e7d2-4fd8-bd43-65a1a07a9049,f0:4a:02:65:d8:d4,{1001},NaN,Network,0.0,70.0,0,0,0


In [40]:
# Filter for rows that have a valid target (category is not null)
ml_df = final_df[final_df['category'].notna()]

# Stricter filtering: Drop rows where ALL features are missing (e.g., no protocol, no bytes)
ml_df = ml_df[
    (ml_df['total_bytes_sent'] > 0) |
    (ml_df['total_bytes_received'] > 0) 
    # (ml_df['protocols_used'] != '')
]

print(f"✅ Number of rows in final_df before filtering: {len(final_df)}")
print(f"✅ Number of rows usable for ML (with valid category): {len(ml_df)}")

display(ml_df)

✅ Number of rows in final_df before filtering: 119125
✅ Number of rows usable for ML (with valid category): 61103


,id,hostname,ipaddress,asset_type_id,mac_address,vlan_ids,parent_asset_id,category,total_bytes_sent,total_bytes_received,SNMP,TCP,UDP
0,93c84f4c-e251-443e-aedb-0f6d08fbbbb0,ec2-3-230-209-14.compute-1.amazonaws.com.,192.168.102.11/24,656c360c-f7e0-4026-b421-771f273958d2,b6:b9:76:54:2d:f6,{2102},0c9d1848-0ee1-4641-a7da-2a7313c45b74,Other,25447457.0,78700306.0,0,1,1
1,1c3225eb-bd10-4065-8528-f8d5f42cff5b,NaN,192.168.45.15/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2102},1fe7e185-f0fb-44bc-a41f-9a357e3ff77f,Network,0.0,29820.0,0,0,0
3,cbc95c38-42fe-473a-bcdb-63869fb7f7d5,NaN,192.168.0.139,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2101},NaN,Network,0.0,2030.0,0,0,0
4,dc032904-bec1-4296-8f84-c06a48fbbe28,DTNBATELIER.sasyb.local.,192.168.103.136/24,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{120},a4c6b641-dcf8-4a1f-a4c6-4b68aac086b8,Network,29788.0,171664.0,0,1,0
5,8fb3c5e7-ee90-4da1-b2c0-e56caaf09669,NaN,192.168.45.68/24,656c360c-f7e0-4026-b421-771f273958d2,5e:0f:5c:dd:26:7f,{45},1d6d0235-84a7-46d8-9d4d-4152e0df0702,Other,839726.0,9083531.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119120,f5efb9ff-9a7c-41de-a527-8fbd8e127f23,NaN,192.168.103.105/24,c8746580-1199-41c3-a659-4870b6d265c9,f8:0d:60:9a:18:10,{2103},NaN,OT,122329.0,9304.0,0,1,1
119121,864c7d2e-6fbb-41e7-a140-b23c4636d789,NaN,10.61.242.238,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{2102},NaN,Network,0.0,1580.0,0,0,0
119122,8bba5974-b250-4b2c-8ba4-53e6b96cfecb,NaN,192.168.1.28,12d5edc6-3b58-404e-93c4-1967b56e3c6d,00:09:0f:09:00:06,{45},NaN,Network,0.0,3150.0,0,0,0
119123,a655da44-5ea3-45f8-838b-b55d65a3197d,NaN,192.168.36.17,962a2cc4-e7d2-4fd8-bd43-65a1a07a9049,f0:4a:02:65:d8:d4,{1001},NaN,Network,0.0,70.0,0,0,0


# Supervised Learning
Since asset_type_id is missing 69.8% for Swiss dataset with 67 distinct id and missing 98.3% for Haarlem dataset with 23 distinct id, I will use 30.2% of Swiss dataset for ML supervised model.

Final labeled dataset size: 63 031 rows
62 343 rows are "Network" category ~98.9%
→ That leaves <700 rows (~1.1%) for all other categories combined: IT, OT, IoT, Other


A **Stratified** split ensures the test set has the same proportions as training set.

In [27]:
from sklearn.model_selection import train_test_split

X = ml_df.drop(columns=['category'])  # features
y = ml_df['category']                # target

# Stratified split: keeps class distribution same in train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Training set class distribution:")
print(y_train.value_counts(normalize=True))
# print(y_train.value_counts(normalize=False))

print("\nTest set class distribution:") 
print(y_test.value_counts(normalize=True))
# print(y_test.value_counts(normalize=False)

Training set class distribution:
category
Network    0.992410
IT         0.003539
Other      0.002782
OT         0.000839
IoT        0.000430
Name: proportion, dtype: float64

Test set class distribution:
category
Network    0.992390
IT         0.003519
Other      0.002782
OT         0.000900
IoT        0.000409
Name: proportion, dtype: float64


# Unused Code

In [ ]:
# protocol_counts = (
#     rel_service_merged
#     .groupby('source_asset_id')['protocol']
#     .value_counts()
#     .unstack(fill_value=0)
#     .add_prefix('protocol_')
# )

# final_df = final_aggregated_df.merge(
#     protocol_counts, 
#     left_on='asset_id', 
#     right_index=True, 
#     how='left'
# ).fillna(0)

# final_df = final_df[[
#     'asset_id', 'source_asset_id', 'target_asset_id', 'relationship_id',
#     'protocol', 'bytes_sent', 'bytes_received', 'count',
#     'hostname', 'ipaddress', 'mac_address', 'vlan_ids', 'parent_asset_id',
#     'asset_type_id', 'asset_type_name', 'category'
# ]]


# THIS PART BELOW IS BYTES NOT AGGREGATED
# sender_df = (
#     rel_service_merged
#     .merge(
#         asset_merged,
#         left_on='source_asset_id',
#         right_on='asset_id',
#         how='left'
#     )
#     .rename(columns={'bytes': 'bytes_sent'})
#     .assign(bytes_received=0)  # Receivers get 0 here
# )

# receiver_df = (
#     rel_service_merged
#     .merge(
#         asset_merged,
#         left_on='target_asset_id',
#         right_on='asset_id',
#         how='left'
#     )
#     .rename(columns={'bytes': 'bytes_received'})
#     .assign(bytes_sent=0)  # Senders get 0 here
# )
# final_df = pd.concat([sender_df, receiver_df])

# Below is old code that was not used in the final version
# # Aggregate all protocols used per source asset
# protocols_df = service_rel.groupby('source_asset_id')['protocol']\
#     .apply(lambda x: ','.join(x.dropna().unique())).reset_index()
# protocols_df = protocols_df.rename(columns={
#     'source_asset_id': 'id',
#     'protocol': 'protocols_used'
# })

# # Merge bytes sent and received into asset_merged
# final_df = asset_merged.merge(bytes_sent_df, on='id', how='left')
# final_df = final_df.merge(bytes_recv_df, on='id', how='left')
# final_df = final_df.merge(protocols_df, on='id', how='left')
# final_df[['total_bytes_sent', 'total_bytes_received']] = final_df[
#     ['total_bytes_sent', 'total_bytes_received']
# ].fillna(0)
# final_df['protocols_used'] = final_df['protocols_used'].fillna('')

# # One-hot encode protocols_used
# protocol_dummies = final_df['protocols_used'].str.get_dummies(sep=',')

# # Merge back into final_df and drop original protocols_used column
# final_df = final_df.drop(columns=['protocols_used']).join(protocol_dummies)

# selected_columns = [
#     'id', 'hostname', 'ipaddress', 'asset_type_id', 
#     'mac_address', 'vlan_ids', 'parent_asset_id',
#     'category', 'total_bytes_sent', 'total_bytes_received'
# ]
# final_df = final_df[selected_columns + list(protocol_dummies.columns)]

# final_df = asset_merged.merge(
#     service_rel_merged

# final_df = final_df[[
#     'asset_id', 'total_bytes_sent', 'total_bytes_received', 
#     'hostname', 'ipaddress', 'mac_address', 'vlan_ids', 'parent_asset_id',
#     'asset_type_id', 'asset_type_name', 'category', 'protocol_used'
# ]+ list(protocol_dummies.columns)]

# display(final_df.columns)
# protocol_used = (
#     rel_service_merged
#     .groupby('source_asset_id')['protocol']
#     .value_counts()
#     .unstack(fill_value=0))

# rel_service_merged = rel_service_merged.merge(
#     protocol_used, 
#     left_on='source_asset_id', 
#     right_index=True, 
#     how='left'
# ).fillna(0) 

# final_aggregated_df = (asset_merged
#     .merge(bytes_sent_df, on='asset_id', how='left')
#     .merge(bytes_recv_df, on='asset_id', how='left')
#     .fillna({
#         'total_bytes_sent': 0,
#         'total_bytes_received': 0
#     }))


# final_df = final_aggregated_df.merge(
#     protocol_counts, 
#     left_on='asset_id', 
#     right_index=True, 
#     how='left'
# ).fillna(0)


# display(final_aggregated_df.columns)
# display(bytes_sent_df)
# display(protocol_used)
# display(rel_service_merged)

# Code Debugging

In [226]:
# check1 = final_df[final_df['id']=='eb2cb00a-5b3e-4ebb-a661-81406babfdc3']
check1 = final_df[final_df['asset_id']=='a88fec76-bbaf-469d-8cd2-3b6b0ae8fc18']

display(check1)

,asset_id,total_bytes_sent,total_bytes_received,hostname,ipaddress,mac_address,vlan_ids,parent_asset_id,asset_type_id,asset_type_name,category,protocols_used,SNMP,TCP,UDP
4,a88fec76-bbaf-469d-8cd2-3b6b0ae8fc18,0.0,0.0,NaN,27.25.152.221,f0:4a:02:65:d8:d4,{1001},NaN,962a2cc4-e7d2-4fd8-bd43-65a1a07a9049,Network asset,Network,,0,0,0


In [229]:
check = s_rela[s_rela['target_asset_id'] == 'a88fec76-bbaf-469d-8cd2-3b6b0ae8fc18']
# check = s_rela[s_rela['source_asset_id'] == 'a88fec76-bbaf-469d-8cd2-3b6b0ae8fc18']

# check = final_df[final_df['id'] == 'eb2cb00a-5b3e-4ebb-a661-81406babfdc3']
# check = s_services[s_services['bytes'] == 174480.0]
# check = s_services[s_services['source_component_id'] == 'eb2cb00a-5b3e-4ebb-a661-81406babfdc3']

display(check)

,id,tags,date_created,date_updated,source_asset_id,target_asset_id
256175,810fab44-e1e4-4eef-84e8-af0f8084d35d,NaN,2024-11-07 06:11:26.278465+01,2024-11-07 06:11:26.278488+01,48dcb782-a6f0-4863-b070-dac5deacb858,a88fec76-bbaf-469d-8cd2-3b6b0ae8fc18


In [232]:
check_id = 'a88fec76-bbaf-469d-8cd2-3b6b0ae8fc18'

# Find all relationship IDs involving asset 'eb2'
eb2_relationships = s_rela[
    (s_rela['source_asset_id'] == check_id) | (s_rela['target_asset_id'] == check_id)
]
eb2_services = s_services[
    (s_services['id'] == check_id) | (s_services['id'] == check_id)
]

# Extract those relationship IDs
eb2_rela_ids = eb2_relationships['id'].unique()

# Find all service entries linked to those relationships
eb2_services = s_services[s_services['relationship_id'].isin(eb2_rela_ids)]
eb2_service_rel = eb2_services.merge(
    s_rela[['id', 'source_asset_id', 'target_asset_id']],
    left_on='relationship_id',
    right_on='id',
    how='left',
    suffixes=('', '_rela')
)

# Filter sent vs. received from eb2
eb2_sent = eb2_service_rel[eb2_service_rel['source_asset_id'] == check_id]
eb2_received = eb2_service_rel[eb2_service_rel['target_asset_id'] == check_id]

# Sum bytes
print("📤 Bytes sent by eb2:", eb2_sent['bytes'].sum())
print("📥 Bytes received by eb2:", eb2_received['bytes'].sum())

print("\n🔍 All service entries involving eb2:")
display(eb2_service_rel[['id', 'relationship_id', 'bytes', 'source_asset_id', 'target_asset_id']])
display(eb2_relationships )
display(eb2_services )

📤 Bytes sent by eb2: 0.0
📥 Bytes received by eb2: 0.0

🔍 All service entries involving eb2:


,id,relationship_id,bytes,source_asset_id,target_asset_id


,id,tags,date_created,date_updated,source_asset_id,target_asset_id
256175,810fab44-e1e4-4eef-84e8-af0f8084d35d,NaN,2024-11-07 06:11:26.278465+01,2024-11-07 06:11:26.278488+01,48dcb782-a6f0-4863-b070-dac5deacb858,a88fec76-bbaf-469d-8cd2-3b6b0ae8fc18


,id,source_port,target_port,protocol,date_created,date_updated,relationship_id,source_component_id,target_component_id,count,bytes


In [30]:
# Distinct type names in swiss
original_type_name_s = s_types['name'].dropna().unique()
print("Original distinct type.name values:")
for name in original_type_name_s:
    print(f"- {name}")
print(f"\nTotal: {len(original_type_name_s)} distinct names\n")


Original distinct type.name values:
- Network asset
- IT asset
- Other asset
- IoT asset
- Nautilus Sensorbox ACS-NAUT-2E
- Nautilus Sensor ACS-NAUTSB-4E2S
- OT asset
- Webserver
- Modbus OT Asset
- Domain Controller (Active Directory)
- Virtualisation

Total: 11 distinct names



In [31]:
# # Distinct type names in haarlem
# original_type_name_h = h_types['name'].dropna().unique()
# print("Original distinct type.name values:")
# for name in original_type_name_h:
#     print(f"- {name}")
# print(f"\nTotal: {len(original_type_name_h)} distinct names\n")


In [32]:
# Transform haarlem type names by removing vendor info (e.g., "IT asset by Dell Inc." -> "IT asset")
# def simplify_name(name):
#     if ' by ' in name:
#         return name.split(' by ')[0].strip()
#     return name.strip()

# haarlem_types['simplified_name'] = haarlem_types['name'].apply(simplify_name)

# # Step 3: New distinct simplified names
# new_names = haarlem_types['simplified_name'].dropna().unique()
# print("Simplified type.name values (e.g., IT asset, OT asset):")
# for name in new_names:
#     print(f"- {name}")
# print(f"\nTotal: {len(new_names)} distinct simplified names")